In [ ]:
# parameters
datadir = ""
scan_numbers_string = ""
all_scans = ""
print_start_time = ""
print_cmd = ""
print_temp = ""
print_energy = ""
print_sx = ""
print_sy = ""
print_sz = ""
other_vars = ""

In [ ]:
import os, re, logging, dominate
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from base64 import b64encode
from io import BytesIO
from dominate.tags import *
from collections import defaultdict
from mmg_toolbox import metadata, Experiment
from mmg_toolbox.utils.misc_functions import string2numbers


In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [ ]:
if not os.path.isdir(datadir):
    raise ValueError("data directory not found")

In [ ]:
all_scans = True if all_scans.lower() == "true" else False
print_start_time = True if print_start_time.lower() == "true" else False
print_cmd = True if print_cmd.lower() == "true" else False
print_temp = True if print_temp.lower() == "true" else False
print_energy = True if print_energy.lower() == "true" else False
print_sx = True if print_sx.lower() == "true" else False
print_sy = True if print_sy.lower() == "true" else False
print_sz = True if print_sz.lower() == "true" else False
other_vars = [s.strip() for s in other_vars.split(',')]

In [ ]:
logger.debug(
f"""
Provided arguments:
     Scan Numbers: {scan_numbers_string}
        All Scans: {all_scans}
 Print Start Time: {print_start_time}
    Print Command: {print_cmd}
Print Temperature: {print_temp}
     Print Energy: {print_energy}
   Print Sample x: {print_sx}
   Print Sample y: {print_sy}
   Print Sample z: {print_sz}
   Other Variable: "{other_vars}"
"""
)

In [ ]:
exp = Experiment(datadir)
if len(exp) == 0:
    raise FileNotFoundError(f"No .nxs files found in {datadir}")

In [ ]:
all_scan_numbers = exp.all_scan_numbers()
if all_scans:
    input_numbers = all_scan_numbers
else:
    input_numbers = string2numbers(scan_numbers_string)
scans = [exp.scans(n)[0] for n in all_scan_numbers if n in input_numbers]

In [ ]:
# %%capture
days = defaultdict(list)
for scan in scans[:10]:
    date = "{:%Y-%m-%d}".format(scan(f"{metadata.start}"))

    fig = plt.figure(figsize=(6, 6),tight_layout=True)
    try:
        ax = scan.plot()
    except Exception as e:
        print(repr(scan), 'Error:')
        print(e)
        ax = plt.axes().annotate(f"Does not work\n{e}", xy=(0.4,0.5))
    fig.axes.append(ax)
    tempfile = BytesIO()
    fig.savefig(tempfile, format='png')
    plt.close(fig)
    encoded = b64encode(tempfile.getvalue()).decode('utf-8')

    scan_info = div(style="display:flex;flex-direction:row;margin:40px")
    with scan_info as d:
        img(src="data:image/png;base64,{}".format(encoded), style="width:40%;margin-right:10px")
        with div(style="font-size:14pt;width:60%") as m:
            m += scan.format("Filename: {filename}")
            m += br()
            if print_start_time:
                m += "Start Time: {:%Y-%m-%d %H:%M}".format(
                    scan(f"{metadata.start}")
                )
                m += br()
            if print_cmd:
                m += "Command: {}".format(scan(f"{metadata.cmd}"))
                m += br()
            if print_temp:
                m += "Temperature: {:.2f}".format(scan(f"{metadata.temp}"))
                m += br()
            if print_energy:
                m += "Beam Energy: {:.3f}".format(scan(f"{metadata.energy}"))
                m += br()
            if print_sx:
                x_string = "mean((ddif_x|scm_x|hfm_x|em_x|manipulator_x|cryo_x|sx))"
                m += "Sample x: {:.3f}".format(scan(x_string))
                m += br()
            if print_sy:
                y_string = "mean((ddif_y|scm_y|hfm_y|em_y|manipulator_y|cryo_y|sy))"
                m += "Sample y: {:.3f}".format(scan(y_string))
                m += br()
            if print_sz:
                z_string = "(ddif_z|scm_z|hfm_z|em_z|manipulator_z|cryo_z|sz)"
                m += "Sample z: {:.3f}".format(scan(z_string))
                m += br()
            for s in other_vars:
                if s == "":
                    continue
                m += "{variable}: {value}".format(variable=s, value=scan(s))
                m += br()

    days[date].append(scan_info)